In [12]:
import pandas as pd

from orion.data import load_signal
from orion import Orion
from orion.data import load_anomalies

from mlprimitives.custom.timeseries_preprocessing import time_segments_aggregate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from mlprimitives.custom.timeseries_preprocessing import rolling_window_sequences
from orion.primitives.timeseries_preprocessing import slice_array_by_dims
from mlprimitives import load_primitive

from orion.evaluation.contextual import contextual_f1_score
import numpy as np

In [5]:
y = np.load('processed/data/y.npy')
index = np.load('processed/data/index.npy')

critic = np.load('processed/tadgan/critic.npy')
y_hat = np.load('processed/tadgan/y_hat.npy')

In [6]:
# computes an array of anomaly scores based on a combination of reconstruction error and critic output
params = {"rec_error_type": "dtw"}

primitive = load_primitive("orion.primitives.tadgan.score_anomalies", 
                           arguments=params)
errors, true_index, true, predictions = primitive.produce(y=y, y_hat=y_hat, critic=critic, index=index)

errors.shape, true_index.shape

Using TensorFlow backend.


In [8]:
print("average error value: {:.2f}".format(errors.mean()))

average error value: 2.34


In [11]:
# extracts anomalies from sequences of errors following the approach
params = {
    "window_size_portion": 0.33, 
    "window_step_size_portion": 0.1,
    "fixed_threshold": True
}

primitive = load_primitive("orion.primitives.timeseries_anomalies.find_anomalies", 
                           arguments=params)
e = primitive.produce(errors=errors, index=true_index)

e.shape

(1, 3)

In [16]:
ground_truth = load_anomalies('S-1')
data_span = (1398168000, 1407823200)
anomalies = [(int(i[0]), int(i[1])) for i in e]

In [18]:
start, end = data_span
contextual_f1_score(ground_truth, anomalies, start=start, end=end, weighted=True)

nan

In [20]:
anomalies

[(1240876800, 1243166400)]

In [21]:
ground_truth

,start,end
0,1398168000,1407823200


In [22]:
ground_truth

,start,end
0,1398168000,1407823200
